In [ ]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.0f' % x)
pd.set_option('display.max_columns', None)
import plotly.express as px

In [ ]:
import tkinter as tk
from tkinter import filedialog

In [ ]:
def get_path():
    root = tk.Tk()
    root.withdraw()
    path = filedialog.askopenfilename()
    return path

In [ ]:
def get_sheet(path):
    df = pd.read_excel(path, None)
    sheet_dict = {}
    for i, j in enumerate(df.keys()):
        print(i, j, sep='\t')
        sheet_dict.setdefault(i, j)
    sheet = eval(input('Please enter the sheet number to open:\n'))
    df = df[sheet_dict[sheet]]
    return df

In [ ]:
path = get_path()

In [ ]:
df = get_sheet(path)

In [ ]:
teltonika = df[(df['狀態'].str.contains('未出')) & (df['Group'] == 'TELTONIKA') & (df['BG'] == 'RF') & (~df['出通單號'].notna()) & (~df['品名'].str.contains('RFDPA'))]

In [ ]:
result = teltonika.groupby([pd.Grouper(key='預交日期', freq='M'), 'Group', '品名', 'Subcategory', '銷售單號', '開單日期'])[['數量']].sum().reset_index()

In [ ]:
result['Month'] = result['預交日期'].dt.month_name()

In [ ]:
result = result[~result['品名'].str.contains('DAM')]

In [ ]:
fig = px.histogram(result, x='Month', y='數量', color='Subcategory', title='TELTONIKA shipment plan in the following months', labels=dict(Subcategory='Type', 數量='Qty.'))

In [ ]:
table = result.style.format({'預交日期': '{:%Y-%m}', '開單日期': '{:%Y-%m-%d}', '數量': '{:,.0f}'}).hide_index().set_caption('TELTONIKA在途訂單各月未交數量')

In [ ]:
with open('Teltonika_shipment_plan.html', 'w') as f:
    f.write(table.render())
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.close()

In [ ]:
result.to_excel(r"C:\Users\kaihsu\Desktop\teltonika_shipment_plan.xlsx", index=False)

In [ ]:
df[(df['Group'] == 'TELTONIKA') & (~df['品名'].str.contains('PA|RFDP|DAM')) & (df['BG'] == 'RF')]['品名'].unique()

In [ ]:
from etl_tools import select_sheet

In [ ]:
df = select_sheet()

In [ ]:
result = df[(df["BG"] == "RF") & (df["Group"] == "TELTONIKA") & (df["狀態"].str.contains("出")) & (df["預交年份"] == 2021) & (df["預交日期"] >= "2021-03-01") & (df["品名"].isin(["RFDPA191300SMTB806J4", "RFDPA171300SBAB823J4", "RFDPA141300SBLB803J4", "RFDPA141300SBLB803J4"]))]

In [ ]:
result = result.pivot_table(index="品名", values="數量", columns="預交日期")

In [ ]:
result = pd.DataFrame(result.to_records())

In [ ]:
result.columns = ['品名', '2021-03-09', '2021-03-26',
       '2021-04-02', '2021-04-26', '2021-05-07']

In [ ]:
result.style.format({"2021-03-09":"{:,.0f}", "2021-03-26":"{:,.0f}", "2021-04-02":"{:,.0f}", "2021-04-26":"{:,.0f}", "2021-05-07":"{:,.0f}"})

In [ ]:
a, b, c = map(int, input("enter somthing").split())

In [ ]:
wip = df[(df["BG"] == "WIP") & (df["狀態"].str.contains("出")) & (df["預交年份"] == 2021) & (df["預交月份"] == "March")]

In [ ]:
wip = wip.pivot_table(index=["負責業務", "Group"], values="本國幣別NTD", aggfunc="sum", margins=True, margins_name="總計").reset_index()

In [ ]:
wip.style.format({"本國幣別NTD":"{:,.0f}"}).hide_index().set_caption("2021年3月份WIP實績")

## 出貨排程確認

In [ ]:
df = pd.read_clipboard()

In [ ]:
weekly = pd.read_excel('/Users/kai/OneDrive/INPAQ/業績總表/Weekly report_v0.1.xlsx', sheet_name='出貨明細')

In [ ]:
weekly = weekly[['銷售單號', '預交日期', '數量', '品名', '銷售項次']]

In [ ]:
weekly.columns = ['訂單單號', '預交日期', '未交量', '品名', '項次']

In [ ]:
result = pd.merge(df, weekly, on=['訂單單號', '未交量', '品名', '項次'], how='left')

In [ ]:
result

In [ ]:
result.to_excel('/Users/kai/Desktop/Susan.xlsx', index=False)